# Week 3 - Segmenting and Clustering Neighborhoods in Toronto


#### Task 1 - Prepare the dataframe

1. import libraries and scrape with beautifulsoup

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
! easy_install beautifulsoup4

Searching for beautifulsoup4
Best match: beautifulsoup4 4.6.0
Adding beautifulsoup4 4.6.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4


In [3]:
from bs4 import BeautifulSoup

In [4]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]

In [5]:
codes_list=[]
borough_list=[]
neighborhood_list=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        codes_list.append(tag.text)
    if i == 2:
        borough_list.append(tag.text)
    if i == 3: 
        neighborhood_list.append(tag.text)
    i = i+1
    if i==4:
        i=1

2. Creating the dataframe

In [13]:
d = {'PostalCode': codes_list, 'Borough': borough_list, 'Neighborhood': neighborhood_list}
tor = pd.DataFrame(data=d,columns=['PostalCode','Borough','Neighborhood'])
tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


3. Cleaning the dataframe: removing "Not assigned" under Borough and remove "\n" values in the Neighbourhood names

In [31]:
tor_b = tor[tor.Borough != 'Not assigned']
tor_b.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [37]:
tor_b['Neighborhood'].replace(to_replace='\n', value='', regex=True, inplace=True)
tor_b.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [38]:
neighborhood.replace(to_replace='Not assigned', value=tor_b['Borough'], inplace=True)

4. Cleaning the dataframe: replacing "Not assigned" under Neighborhood with the Borough name

In [39]:
tor_b['Neighborhood'].replace(to_replace='Not assigned', value=tor_b['Borough'], inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


5. Combining Neighborhood names with same Postal Code and separating with ,

In [41]:
tor_j = tor_b.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(lambda x: ', '.join(x))
tor_j.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


6. Shape output of dataframe

In [42]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(tor_j['Borough'].unique()),
        tor_j.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


In [43]:
tor_j.shape[0]

103